# Curva Mercado Local

In [1]:
import autograd.numpy as np
from autograd import grad
from functools import partial
from bisect import bisect_right

**Primer Paso del Bootstrapping Local**

- Calculo E(ICP6M) con la cotización del swap a 6M
- Tengo las cotizaciones de swap y basis a 1Y:
  - Basis: $V_{usd}=PV(discount_{clp},proyeccion_{clp})$
  
  - Swap: $PV_{fija}(discount_{clp})=PV(discount_{clp},proyeccion_{clp})$
  
  - Entonces: $PV_{fija}(discount_{clp})=V_{usd}$
  
  - Finalmente: Resuelvo la ecuación del basis usando la curva de descuento que acabo de obtener. Esta ecuación es de 1 incógnita porque ya tengo el E(ICP6M).

In [2]:
swap_6m = .000495
v_usd = 101.0
swap_1y = .000515
basis_1y = .00032

In [3]:
def lin_interpol(tenors, rates, tenor):
    if tenor >= tenors[len(tenors) - 1]:
        return rates[len(tenors) - 1]
    elif tenor <= tenors[0]:
        return rates[0]
    else:
        i = bisect_right(tenors, tenor) - 1
        m = (rates[i + 1] - rates[i]) / (tenors[i + 1] - tenors[i])
        return rates[i] + m * (tenor - tenors[i])
    
def df(rate, tenor):
    return (1 + rate)**(-tenor / 365.0)

def present_value(interp, disc, cashflow, tenor, tenors, rates):
    rate = interp(tenors, rates, tenor)
    return df(rate, tenor) * cashflow

def kron(i, j):
    return int(i == j)

def dfs(rates, tenors):
    return np.array([df(z[0], z[1]) for z in zip(rates, tenors)])

def lin_interpols(tenors, rates, new_tenors):
    return np.array([lin_interpol(tenors, rates, t) for t in new_tenors])

def fixed_rate_leg(nocional, tasa, num_cupones):
    return np.array([.5 * 365.0 * i for i in range(1, num_cupones + 1)]), \
        np.array([nocional * (kron(i, num_cupones) + tasa / 2.0)
                  for i in range(1, num_cupones + 1)])

def present_value_2(interp, disc, cashflows_tenors, cashflows, curve_tenors, curve_rates):
    cashflow_rates = interp(curve_tenors, curve_rates, cashflows_tenors)
    return np.dot(dfs(cashflow_rates, cashflows_tenors), cashflows)

def pv_fixed_leg(interp, disc, nocional, tasa, num_cupones, curve_tenors, curve_rates):
    plazos, flujos = fixed_rate_leg(nocional, tasa, num_cupones)
    return present_value_2(interp, disc, plazos, flujos, curve_tenors, curve_rates)

def solve_2(nocional, tasa, num_cupones, obj, tenors, rates, pv):
    epsilon = .000001
    g = grad(pv, argnum=4)
    rates_ = np.array([r for r in rates] + [0.0,])
    print(rates_)
    which = len(rates_) - 1
    print(f'which: {which}')
    delta = 1
    while delta > epsilon:
        q = (pv(nocional, tasa, num_cupones, tenors, rates_) - obj)
        q /= g(nocional, tasa, num_cupones, tenors, rates_)[which]
        r1 = rates_[which] - q
        print(r1)
        delta = abs(r1 - rates_[which])
        print(f'delta: {delta}')
        if type(r1) is np.float64:
            rates_ = np.array([r for r in rates] + [r1,])
        else:
            rates_ = np.array([r for r in rates] + [r1._value,])
    return r1

In [4]:
tenors0 = np.array([1.0, 365.0])
rates0 = np.array([.005,])

In [5]:
pv = partial(pv_fixed_leg, lin_interpols, dfs)

In [6]:
solve_2(100.0, swap_1y, 2, 100.0, tenors0, rates0, pv)

[0.005 0.   ]
which: 1
0.0005145124524150278
delta: 0.0005145124524150278
0.0005147773010557074
delta: 2.6484864067957605e-07


0.0005147773010557074